# Evaluating QA: Metrics, Predictions, and the Null Response
> A deep dive into computing QA predictions and when to tell BERT to zip it! 

- toc: true
- badges: true
- comments: false
- categories: [PyTorch, Hugging Face, Wikipedia, BERT, Transformers]

### Prerequisites
* A basic understanding of Transformers and PyTorch
* A Transformer fine-tuned on SQuAD2.0
* The SQuAD2.0 dev set

### What you'll learn
* Metrics for evaluating QA performance
* How to evaluate BERT on SQuAD2.0
* How to handle the Null Response -- when a question doesn't have answer in the passage
* Implementing a more robust answering method for your QA system

In our last post, [Building a QA System with BERT on Wikipedia](https://qa.fastforwardlabs.com/pytorch/hugging%20face/wikipedia/bert/transformers/2020/05/19/Getting_Started_with_QA.html), we used the HuggingFace framework to train BERT on the SQuAD2.0 dataset and built a simple QA system on top of the Wikipedia search engine. This time, we'll look at the quality of BERT for Question Answering. We'll cover what metrics are used to quantify quality, how to evaluate BERT with the Hugging Face framework and the importance of the "null response" -- when a question does not have an answer -- for both improved performance and more realistic QA output. 


# Answering questions is complicated
Quantifying the success of answering a question is a tricky task. When you or I ask a question, the correct answer could take multiple forms. Let's look at an example. 

In our previous post, BERT answered the question, "Why is the sky blue?" with "Rayleigh scattering" but another correct answer would be 

"The Earth's atmosphere scatters short-wavelength light more efficiently than that of longer wavelengths. Because its wavelengths are shorter, blue light is more strongly scattered than the longer-wavelength lights, red or green. Hence the result that when looking at the sky away from the direct incident sunlight, the human eye perceives the sky to be blue." 

Both of these passages can be found in the Wikipedia article [Diffuse Sky Radiation](https://en.wikipedia.org/wiki/Diffuse_sky_radiation) and both are correct. However, I've also had a model return "because its wavelengths are shorter" as the answer, which is close but not really a correct answer: the sky itself doesn't have a wavelength -- this answer is missing too much context to be useful. 

How should we judge a model when there can be multiple correct answers and even more incorrect answers? To properly assess the quality of a model we need a gold standard set of questions and answers! Let's turn back to the SQuAD dataset.

# More about SQuAD
The SQuAD dataset comes in two flavors: SQuAD1.1 and SQuAD2.0. The latter contains the same questions and answers as the former but also includes additional questions that _cannot_ be answered by the accompanying passage. This is intended to provide a more realistic question answering task because often times there really won't be an answer to the question in the document we're parsing. This ability to properly identify when a question does not have an answer is much more challenging for Transformer models and it's why we focused on this dataset rather than SQuAD1.1. 

SQuAD2.0 consists of more than 130k questions, of which a full third do not have an answer that can be found in the associated passage. The dev set in particular contains a 50/50 split of answerable questions. SQuAD examples consist of question - context pairs. The context is a single paragraph from a Wikipedia article. Each paragraph will have several questions (both answerable and unanswerable) associated with it. Paragraphs are drawn from 35 Wikipedia articles. Every paragraph in the article has at least one question associated with it. The full SQuAD stats are shown below from the paper XXX. 

![](my_icons/squad_datasets.png "SQuAD Stats")

Last time we fine-tuned the model on the SQuAD2.0 train set. This time we'll evaluate the model on the SQuAD2.0 dev set.


Use the hidden cells below to get set set up, if needed.

In [ ]:
# collapse-hide

# use this cell to install packages if needed
!pip install torch  torchvision -f https://download.pytorch.org/whl/torch_stable.html
!pip install transformers

In [163]:
# collapse-hide
import json
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModelForQuestionAnswering

# This is the directory in which we'll store all evaluation output
model_dir = "models/distilbert/twmkn9_distilbert-base-uncased-squad2/"

In [ ]:
# collapse-hide

# Download the SQuAD2.0 dev set
!wget -P data/squad/ https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v2.0.json

### Load the SQuAD 2.0 dev set using HF data processors

HuggingFace provide the [Processors](https://huggingface.co/transformers/main_classes/processors.html) library for fascilitating basic processing tasks with some canonical NLP datasets. The processors can be used for loading datasets and converting their examples to features for direct use in the model. We'll be using the [SQuAD processors](https://huggingface.co/transformers/main_classes/processors.html#squad). 

In [157]:
from transformers.data.processors.squad import SquadV2Processor

# This processor loads the SQuAD2.0 dev set examples
processor = SquadV2Processor()
examples = processor.get_dev_examples("data/squad/", filename="dev-v2.0.json")
print(len(examples))

100%|██████████| 35/35 [00:05<00:00,  6.57it/s]

11873


While `examples` is a list, most other tasks we'll work with use a unique identifier - one for each question in the dev set. 

In [8]:
# generate some maps to help us identify examples of interest
qid_to_example_index = {example.qas_id: i for i, example in enumerate(examples)}
qid_to_has_answer = {example.qas_id: bool(example.answers) for example in examples}
answer_qids = [qas_id for qas_id, has_answer in qid_to_has_answer.items() if has_answer]
no_answer_qids = [qas_id for qas_id, has_answer in qid_to_has_answer.items() if not has_answer]

In [9]:
def display_example(qid):    
    idx = qid_to_example_index[qid]
    q = examples[idx].question_text
    c = examples[idx].context_text
    a = [answer['text'] for answer in examples[idx].answers]
    
    print(f'Example {idx} of {len(examples)}\n---------------------')
    print(f"Q: {q}\n")
    print(f"Context: \n{c}\n")
    print(f"True Answers:\n{a}")

#### A SQuAD example 

Approximately 50% of the examples in the dev set are questions that have answers contained within their corresponding passage. In these cases, up to five possible correct answers are provided (questions and answers were generated and identified by crowd-sourced workers). Answers must be direct excerpts from the passage but we can see there are several ways to have a "correct" answer. 

In [10]:
display_example(answer_qids[1300])

Example 2548 of 11873
---------------------
Q: Where on Earth is free oxygen found?

Context: 
Free oxygen also occurs in solution in the world's water bodies. The increased solubility of O
2 at lower temperatures (see Physical properties) has important implications for ocean life, as polar oceans support a much higher density of life due to their higher oxygen content. Water polluted with plant nutrients such as nitrates or phosphates may stimulate growth of algae by a process called eutrophication and the decay of these organisms and other biomaterials may reduce amounts of O
2 in eutrophic water bodies. Scientists assess this aspect of water quality by measuring the water's biochemical oxygen demand, or the amount of O
2 needed to restore it to a normal concentration.

True Answers:
['water', "in solution in the world's water bodies", "the world's water bodies"]


#### A SQuAD negative example

The other half of the questions in dev set do not have an answer in the corresponding passage. These questions were generated by crowd-sourced workers to be related and relevant to the passage but unanswerable by that passage. There are thus no True Answers associated with these questions as we see in the example below. 

Note: In this case, the question is a trick -- the numbers are reoriented in a way that no longer holds true. Will the model pick up on that? 

In [122]:
display_example(no_answer_qids[1254])

Example 2564 of 11873
---------------------
Q: What happened 3.7-2 billion years ago?

Context: 
Free oxygen gas was almost nonexistent in Earth's atmosphere before photosynthetic archaea and bacteria evolved, probably about 3.5 billion years ago. Free oxygen first appeared in significant quantities during the Paleoproterozoic eon (between 3.0 and 2.3 billion years ago). For the first billion years, any free oxygen produced by these organisms combined with dissolved iron in the oceans to form banded iron formations. When such oxygen sinks became saturated, free oxygen began to outgas from the oceans 3–2.7 billion years ago, reaching 10% of its present level around 1.7 billion years ago.

True Answers:
[]


# Metrics for QA

There are two dominant metrics used by many question answering datasets: exact match (EM) and F1 score. These scores are computed on individual question+answer pairs. When multiple correct answers are possible for a given question (as in the previous example), the maximum score over all possible correct answers is computed. A model is given overall EM and F1 scores by averaging over the individual example scores.  


### Exact Match
This metric is as simple as it sounds. For each question+answer pair, if the characters of the model's prediction exactly match the characters of the known answer, EM = 1, otherwise EM = 0. This is a strict all-or-nothing metric; being off by a single character would result in a score of 0 for that prediciton. When assessing against a negative example, if the model predicts any text at all it automatically receives a 0 for that example. 

### F1 
This metric can be found in many analyses so it shouldn't be a surprise to see here. F1 is the harmonic mean of the precision and recall. In this case, it's computed over the individual words in the prediction against those in the true answer. The number of shared words provides the basis of the f1 score.


![](my_icons/f1score.png "F1 score")

Let's see how these metrics work in practice. We'll load up a fine-tuned model and its tokenizer and compare our predictions against the True Answers. 

### Load a Transformer model fine-tuned on SQuAD 2.0

In [17]:
tokenizer = AutoTokenizer.from_pretrained("twmkn9/distilbert-base-uncased-squad2")
model = AutoModelForQuestionAnswering.from_pretrained("twmkn9/distilbert-base-uncased-squad2")

The following `get_prediction` method is essentially identical to what we used last time in our simple QA system.

In [124]:
def get_prediction(qid):
    # given a question id (qas_id or qid), load the example, get the model outputs and generate an answer
    question = examples[qid_to_example_index[qid]].question_text
    context = examples[qid_to_example_index[qid]].context_text

    inputs = tokenizer.encode_plus(question, context, return_tensors='pt')

    outputs = model(**inputs)
    answer_start = torch.argmax(outputs[0])  # Get the most likely beginning of answer with the argmax of the score
    answer_end = torch.argmax(outputs[1]) + 1 

    answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(inputs['input_ids'][0][answer_start:answer_end]))

    return answer

Below are some functions we'll need to compute our quality metrics. 

In [19]:
# These functions are heavily influenced by the HF squad_metrics.py script
def normalize_text(s):
    """Removing articles and punctuation, and standardizing whitespace are all typical text processing steps."""
    import string, re

    def remove_articles(text):
        regex = re.compile(r"\b(a|an|the)\b", re.UNICODE)
        return re.sub(regex, " ", text)

    def white_space_fix(text):
        return " ".join(text.split())

    def remove_punc(text):
        exclude = set(string.punctuation)
        return "".join(ch for ch in text if ch not in exclude)

    def lower(text):
        return text.lower()

    return white_space_fix(remove_articles(remove_punc(lower(s))))

def compute_exact_match(prediction, truth):
    return int(normalize_text(prediction) == normalize_text(truth))

def compute_f1(prediction, truth):
    pred_tokens = normalize_text(prediction).split()
    truth_tokens = normalize_text(truth).split()
    
    # if either the prediction or the truth is no-answer then f1 = 1 if they agree, 0 otherwise
    if len(pred_tokens) == 0 or len(truth_tokens) == 0:
        return int(pred_tokens == truth_tokens)
    
    common_tokens = set(pred_tokens) & set(truth_tokens)
    
    # if there are no common tokens then f1 = 0
    if len(common_tokens) == 0:
        return 0
    
    prec = len(common_tokens) / len(pred_tokens)
    rec = len(common_tokens) / len(truth_tokens)
    
    return 2 * (prec * rec) / (prec + rec)

def get_gold_answers(example):
    """helper function that retrieves all possible true answers from a squad2.0 example"""
    
    gold_answers = [answer["text"] for answer in example.answers if answer["text"]]

    # if gold_answers doesn't exist it's because this is a negative example - 
    # the only correct answer is an empty string
    if not gold_answers:
        gold_answers = [""]
        
    return gold_answers

In the following cell, we start by computing EM and F1 for our first example - the one that has several True Answers associated with it. 

In [24]:
prediction = get_prediction(answer_qids[1300])
example = examples[qid_to_example_index[answer_qids[1300]]]

gold_answers = get_gold_answers(example)

em_score = max((compute_exact_match(prediction, answer)) for answer in gold_answers)
f1_score = max((compute_f1(prediction, answer)) for answer in gold_answers)

print(f"Question: {example.question_text}")
print(f"Prediction: {prediction}")
print(f"True Answers: {gold_answers}")
print(f"EM: {em_score} \t F1: {f1_score}")

Question: Where on Earth is free oxygen found?
Prediction: water bodies
True Answers: ['water', "in solution in the world's water bodies", "the world's water bodies"]
EM: 0 	 F1: 0.8


We see that our prediction is actually quite close to some of the True Answers resulting in a respectable F1 score. However, it does not exactly match any of them so our EM score is 0. 

Let's try with our negative example now. 

In [125]:
prediction = get_prediction(no_answer_qids[1254])
example = examples[qid_to_example_index[no_answer_qids[1254]]]

gold_answers = get_gold_answers(example)

em_score = max((compute_exact_match(prediction, answer)) for answer in gold_answers)
f1_score = max((compute_f1(prediction, answer)) for answer in gold_answers)

print(f"Question: {example.question_text}")
print(f"Prediction: {prediction}")
print(f"True Answers: {gold_answers}")
print(f"EM: {em_score} \t F1: {f1_score}")

Question: What happened 3.7-2 billion years ago?
Prediction: [CLS] what happened 3 . 7 - 2 billion years ago ? [SEP] free oxygen gas was almost nonexistent in earth ' s atmosphere before photosynthetic archaea and bacteria evolved , probably about 3 . 5 billion years ago . free oxygen first appeared in significant quantities during the paleoproterozoic eon ( between 3 . 0 and 2 . 3 billion years ago ) . for the first billion years , any free oxygen produced by these organisms combined with dissolved iron in the oceans to form banded iron formations . when such oxygen sinks became saturated , free oxygen began to outgas from the oceans
True Answers: ['']
EM: 0 	 F1: 0


Wow. Both or metrics are zero because this model does not correctly asses that this question is unanswerable! Even worse, it seems to have catastrophically failed, including the entire question as part of the answer.  

Now that we have the basics of computing QA metrics on a couple of examples, we need to assess the model on the entire dev set. Luckily, there's a script for that. 

# Evaluating a model on the SQuAD2.0 dev set with HF

The same `run_squad.py` script we used to fine-tune a Transformer for question answering can also be used to evaluate the model! Below are the arguments you'll need to properly evaluate a fine-tuned model for question answering on the SQuAD dev set. Because we using SQuAD2.0 it is **crucial** that you include the `--version_2_with_negative` flag!

In [103]:
!python run_squad.py  \
    --model_type distilbert   \
    --model_name_or_path twmkn9/distilbert-base-uncased-squad2  \
    --output_dir models/distilbert/twmkn9_distilbert-base-uncased-squad2 \
    --data_dir data/squad   \
    --predict_file dev-v2.0.json   \
    --do_eval   \
    --version_2_with_negative \
    --do_lower_case  \
    --per_gpu_eval_batch_size 12   \
    --max_seq_length 384   \
    --doc_stride 128


2020-06-02 19:14:41.314546: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libnvinfer.so.6'; dlerror: libnvinfer.so.6: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:
2020-06-02 19:14:41.314635: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libnvinfer_plugin.so.6'; dlerror: libnvinfer_plugin.so.6: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:
2020-06-02 19:14:41.314648: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:30] Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.
06/02/2020 19:14:42 - WARNING - __main__ -   Process rank: -1, device: cuda, n_gpu: 1, distributed training: False, 16-bits training: False
06/02/2020 19:14:42 - INFO - transformers.configuration_uti

06/02/2020 19:14:50 - INFO - __main__ -   Creating features from dataset file at data/squad
convert squad examples to features: 100%|█| 11873/11873 [02:02<00:00, 97.08it/s]
add example index and unique id: 100%|█| 11873/11873 [00:00<00:00, 461404.92it/s
06/02/2020 19:16:59 - INFO - __main__ -   Saving features into cached file data/squad/cached_dev_distilbert-base-uncased-squad2_384
06/02/2020 19:17:20 - INFO - __main__ -   ***** Running evaluation  *****
06/02/2020 19:17:20 - INFO - __main__ -     Num examples = 12232
06/02/2020 19:17:20 - INFO - __main__ -     Batch size = 12
Evaluating: 100%|███████████████████████████| 1020/1020 [02:23<00:00,  7.13it/s]
06/02/2020 19:19:43 - INFO - __main__ -     Evaluation done in total 143.126918 secs (0.011701 sec per example)
06/02/2020 19:19:43 - INFO - transformers.data.metrics.squad_metrics -   Writing predictions to: models/distilbert/twmkn9_distilbert-base-uncased-squad2/predictions_.json
06/02/2020 19:19:43 - INFO - transformers.data.metr

We've evaluated a `distilbert` model that was fine-tuned on SQuAD2.0 by a member of the NLP community. When running the evaluation we see a number of steps performed: 

1. the dev set is loaded from disk 
2. the examples are converted to features that can be directly fed to the model
3. these features are cached to disk
4. Evaluation proceeds in batches of 12 and finishes in about 2.5 minutes (this is because distilBERT is much faster and more lightweight than BERT)
5. a slew of prediction outputs are written to disk
6. Overall model results are displayed

We'll focus on this last piece of information: the overall model results. 

In [26]:
Results = {
    # a) Scores averaged over all examples in the dev set
    'exact': 66.25958056093658,         
    'f1': 69.66994428499025,            
    'total': 11873,  # number of examples in the dev set
    
    # b) Scores averaged over only positive examples (have answers)
    'HasAns_exact': 68.91025641025641,  
    'HasAns_f1': 75.74076391627662,     
    'HasAns_total': 5928, # number of positive examples
    
    # c) Scores averaged over only negative examples (no answers)
    'NoAns_exact': 63.61648444070648, 
    'NoAns_f1': 63.61648444070648, 
    'NoAns_total': 5945, # number of negative examples
    
    # d) Given probabilities of no-answer for each example, what would the best scores and thresholds be?
    'best_exact': 66.25958056093658, 
    'best_exact_thresh': 0.0, 
    'best_f1': 69.66994428499046, 
    'best_f1_thresh': 0.0
}

The first three blocks of the `Results` output are pretty straightforward. EM and F1 scores are reported over a) the full dev set, b) the set of positive examples, and c) the set of negative examples. This can give you some insight into whether your model is performing adequately on both answer and no-answer questions (this particular model is pretty bad at no-answer questions). 

However, what's going on with that fourth block? This portion of the output is not useful unless you supply the evaluation method with additional information. And for that we'll need to dig a bit deeper into the evaluation process because it turns out that we need to compute more than just a prediction for an answer - we must also compute a prediction for NO answer and we must score both predictions!

The following section will walk through the details of computing robust predictions on SQuAD2.0 examples, including how to score an answer and the null answer, and how to determine which one should be the "correct" prediction for a given example. Feel free to jump ahead to the next section if you want to get to the punchline, however, for those of you considering building a QA system of your own, this information will be invaluable for proper prediction and assessment. 


### Computing predictions [highly technical]

When the tokenized question+context is passed to the model, the output consists of two sets of logits: one for the start of the answer span, the other for the end of the answer span. These logits represent the likelihood of any given token being the start or end of the answer. Every token passed to the model is assigned a logit, including special tokens (e.g, [CLS], [SEP]), and tokens corresponding to the question itself.  

Let's walk through the process using our last example (Q: Why is the theory of evolution so complex?). 

In [126]:
inputs = tokenizer.encode_plus(example.question_text, example.context_text, return_tensors='pt')
start_logits, end_logits = model(**inputs)

In [127]:
# Look at how large the logit is in the [CLS] position (index 0)!  
# Strong possibility that this question has no answer... but our prediction returned an answer anyway!
start_logits

tensor([[  6.4914,  -9.1416,  -8.4068,  -7.5684,  -9.9081,  -9.4256, -10.1625,
          -9.2579, -10.0554,  -9.9653,  -9.2002,  -8.8657,  -9.1162,   0.6481,
          -2.5947,  -4.5072,  -8.1189,  -6.5871,  -5.8973, -10.8619, -11.0953,
         -10.2294,  -9.3660,  -7.6017, -10.8009, -10.8197,  -6.1258,  -8.3507,
          -4.2463, -10.0987, -10.2659,  -8.8490,  -6.7346,  -8.6513,  -9.7573,
          -5.7496,  -5.5851,  -8.9483,  -7.0652,  -6.1369,  -5.7810,  -9.4366,
          -8.7670,  -9.6743,  -9.7446,  -7.7905,  -7.4541,  -1.5963,  -3.8540,
          -7.3450,  -8.1854,  -9.5566,  -8.3416,  -8.9553,  -8.3144,  -6.4132,
          -4.2285,  -9.4427,  -9.5111,  -9.2931,  -8.9154,  -9.3930,  -8.2111,
          -8.9774,  -9.0274,  -7.2652,  -7.4511,  -9.8597,  -9.5869,  -9.9735,
          -7.0526,  -9.7560,  -8.7788,  -9.5117,  -9.6391,  -8.6487,  -9.5994,
          -7.8213,  -5.1754,  -4.3561,  -4.3913,  -7.8499,  -7.7522,  -8.9651,
          -3.5229,  -0.8312,  -2.7668,  -7.9180, -10

In our simple QA system, we predicted the best answer by selecting the start and end tokens with the largest logits, but that's not very robust. In fact, the original BERT paper suggested considering any sensible start+end combination as a possible answer to the question. These combinations would then be scored, and the one with the highest score would be considered the best answer. A possible (candidate) answer is scored as the sum of it's start and end logits. Let's see it in practice. 


To mimic this behavior we'll start by taking the _n_ largest `start_logits` and the _n_ largest `end_logits` as candidates. Any sensible combination of these start + end tokens is considered a candidate answer, however, several consistency checks must first be performed. For example, an answer wherein the end token falls _before_ the start token should be excluded because that just doesn't make sense. Candidate answers wherein the start or end tokens are associated with question tokens are also excluded because the answer to the question should obviously not be in the question itself! It is important to note that the [CLS] token and its corresonding logits are **not** removed because this token indicates the null answer. 

In [128]:
# We can sort our list of start_logits by logit score and keep track of which token they're associated with
def to_list(tensor):
    return tensor.detach().cpu().tolist()

# convert our start and end logit tensors to lists
start_logits = to_list(start_logits)[0]
end_logits = to_list(end_logits)[0]

# sort our start and end logits from largest to smallest, keeping track of the index
start_idx_and_logit = sorted(enumerate(start_logits), key=lambda x: x[1], reverse=True)
end_idx_and_logit = sorted(enumerate(end_logits), key=lambda x: x[1], reverse=True)

# select the top n (in this case, 5)
print(start_idx_and_logit[:5])
print(end_idx_and_logit[:5]) 

[(0, 6.491387367248535), (111, 6.451895713806152), (129, 4.397505760192871), (115, 3.354909658432007), (106, 3.1457457542419434)]
[(119, 6.33292293548584), (0, 6.084450721740723), (135, 4.417276382446289), (116, 4.3764214515686035), (112, 4.125303268432617)]


The null answer token (index 0) is in the top five of both the start and end logit lists. 

In order to eventually predict a text answer (or empty string), we need to keep track of the indexes which can be used to pull the corresponding token ids later on. We'll also need to identify which indexes correspond to the question tokens so that we can ensure we don't allow a nonsensical prediction. 

In [129]:
start_indexes = [idx for idx, logit in start_idx_and_logit[:5]]
end_indexes = [idx for idx, logit in end_idx_and_logit[:5]]

# convert the token ids from a tensor to a list
tokens = to_list(inputs['input_ids'])[0]

# question tokens are defined as those between the CLS token (101, at position 0) and first SEP (102) token 
question_indexes = [i+1 for i, token in enumerate(tokens[1:tokens.index(102)])]
question_indexes

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]

Next, we'll loop through all combinations of the start and end token indexes, excluding nonsensical combinations. We'll save candidate predictions to a list for the next step.

In [130]:
import collections

# keep track of all preliminary predictions
PrelimPrediction = collections.namedtuple(  # pylint: disable=invalid-name
    "PrelimPrediction", ["start_index", "end_index", "start_logit", "end_logit"]
)

prelim_preds = []
for start_index in start_indexes:
    for end_index in end_indexes:
        # throw out invalid predictions
        if start_index in question_indexes:
            continue
        if end_index in question_indexes:
            continue
        if end_index < start_index:
            continue
        prelim_preds.append(
            PrelimPrediction(
                start_index = start_index,
                end_index = end_index,
                start_logit = start_logits[start_index],
                end_logit = end_logits[end_index]
            )
        )

With a list of sensible candidate predictions, it's time to score them. 

For a candidate answer, score = `start_logit` + `end_logit`. Below we sort our candidate predictions by their score.

In [131]:
# sort preliminary predictions by their score
prelim_preds = sorted(prelim_preds, key=lambda x: (x.start_logit + x.end_logit), reverse=True)
prelim_preds

[PrelimPrediction(start_index=0, end_index=119, start_logit=6.491387367248535, end_logit=6.33292293548584),
 PrelimPrediction(start_index=111, end_index=119, start_logit=6.451895713806152, end_logit=6.33292293548584),
 PrelimPrediction(start_index=0, end_index=0, start_logit=6.491387367248535, end_logit=6.084450721740723),
 PrelimPrediction(start_index=0, end_index=135, start_logit=6.491387367248535, end_logit=4.417276382446289),
 PrelimPrediction(start_index=111, end_index=135, start_logit=6.451895713806152, end_logit=4.417276382446289),
 PrelimPrediction(start_index=0, end_index=116, start_logit=6.491387367248535, end_logit=4.3764214515686035),
 PrelimPrediction(start_index=111, end_index=116, start_logit=6.451895713806152, end_logit=4.3764214515686035),
 PrelimPrediction(start_index=0, end_index=112, start_logit=6.491387367248535, end_logit=4.125303268432617),
 PrelimPrediction(start_index=111, end_index=112, start_logit=6.451895713806152, end_logit=4.125303268432617),
 PrelimPredic

Next we need to convert our preliminary predictions into actual text (or the empty string if null). We'll also trim this list down to the best 5 predictions. 

In [134]:
# keep track of all best predictions
BestPrediction = collections.namedtuple(  # pylint: disable=invalid-name
    "BestPrediction", ["text", "start_logit", "end_logit"]
)

nbest = []
seen_predictions = []
for pred in prelim_preds:
    
    # For now we only care about the top 5 best predictions
    if len(nbest) >= 5: 
        break
        
    # loop through predictions according to their start index
    if pred.start_index > 0: # non-null answers have start_index > 0

        text = tokenizer.convert_tokens_to_string(
            tokenizer.convert_ids_to_tokens(
                tokens[pred.start_index:pred.end_index+1]
            )
        )
        # Clean whitespace
        text = text.strip()
        text = " ".join(text.split())

    if text in seen_predictions:
        continue
        
    # flag this text as being seen -- if we see it again, no need to add it to the list
    seen_predictions.append(text) 
    
    # add this text prediction to a pruned list of the top 5 best predictions
    nbest.append(BestPrediction(text=text, start_logit=pred.start_logit, end_logit=pred.end_logit))


# And don't forget -- include the null answer!
nbest.append(BestPrediction(text="", start_logit=start_logits[0], end_logit=end_logits[0]))

The null answer is scored as the sum of the  start_logit and end_logit associated with the [CLS] token. 

 
At this point, we have a neat list of the top 5 best predictions for this question. The number of best predictions for each example is adjustable with the `--n_best_size` argument of the `run_squad.py` script.  The `nbest` predictions for _every question_ in the dev set are saved to disk under `nbest_predictions_.json` in `--output_dir`. This is a great resource if you need to dig into how your model is behaving. Let's take a look at our `nbest` predictions.

In [135]:
nbest

[BestPrediction(text='free oxygen', start_logit=6.491387367248535, end_logit=6.33292293548584),
 BestPrediction(text='free oxygen began to outgas from the oceans', start_logit=6.451895713806152, end_logit=6.33292293548584),
 BestPrediction(text='free oxygen began to outgas from the oceans 3 – 2 . 7 billion years ago , reaching 10 % of its present level', start_logit=6.451895713806152, end_logit=4.417276382446289),
 BestPrediction(text='free oxygen began to outgas', start_logit=6.451895713806152, end_logit=4.3764214515686035),
 BestPrediction(text='outgas from the oceans', start_logit=3.354909658432007, end_logit=6.33292293548584),
 BestPrediction(text='', start_logit=6.491387367248535, end_logit=6.084450721740723)]

TODO: UPDATE THIS PARAGRAPH
Sure enough, we see that the best prediction is identical to the one we originally had: "uncertainties of fossilization". In this case, the best answer indexes didn't yield a nonsensical prediction so taking the max(`start_logit`) and the max(`end_logit`) wasn't an issue. However, we know it's still incorrect. Let's keep going. 


The last step is to compute the the null score -- more specifically, the difference between the null score and the best non-null score as shown below. 

In [136]:
# compute the null score as the sum of the [CLS] token logits
score_null = start_logits[0] + end_logits[0]

# compute the difference between the null score and the best non-null score
score_diff = score_null - nbest[0].start_logit - nbest[0].end_logit

score_diff

-0.2484722137451172

This `score_diff` is computed for every example in the dev set and these scores are saved to disk in the `null_odds_.json`. Let's pull up the score stored for the example we're using and see how we did!

In [158]:
filename = model_dir + 'null_odds_.json'
null_odds = json.load(open(filename, 'rb'))

null_odds[example.qas_id]

-0.2090005874633789

We're a few hundredths of a logit off but not too bad. (In the full HF version there even more checks and layers of sophistication which I have stripped here.)

### The `--null_score_diff_threshold`

In the previous section we computed robust prediction scores for both the best possible text answer and the null answer for questions in the SQuAD2.0 dev set. We also computed the difference between these scores and learned that `run_squad.py` saves these score differences for each example in the `null_odds_.json` file. With can now start to make sense of the fourth block of the results output!

According to the original BERT paper, 

> We predict a non-null answer when sˆi,j > s_null + τ , where the threshold τ is selected on the dev set to maximize F1.  

In other words, the authors are saying that one should predict a null answer for a given example if that example's score difference is above a certain threshold. What should that threshold be? How should we compute it? They give us a recipe: fine tune the score to maximize F1. 

Rather then rerunning `run_squad.py`, we can import the specific method that computes SQuAD evalutation, the aptly named `squad_evaluate`. You can take a look at the code for yourself [here](https://github.com/huggingface/transformers/blob/5856999a9f2926923f037ecd8d27b8058bcf9dae/src/transformers/data/metrics/squad_metrics.py#L211-L239). To use it we'll need 

* the original examples (because that's where the True Answers are stored),
* the predictions (which we already computed when we evaluated using `run_squad.py`), 
* the `null_odds_.json` (also already computed),
* and a threshold. 

In [50]:
from transformers.data.metrics.squad_metrics import squad_evaluate

In [52]:
# Load the predictions we generated earlier
filename = model_dir + 'predictions_.json'
preds = json.load(open(filename, 'rb'))

# Load the null score differences we generated earlier
filename = model_dir = 'null_odds_.json'
null_odds = json.load(open(filename, 'rb'))

Let's re-evaluate our model on SQuAD2.0 using the `squad_evaluate` method. This method uses the score differences for each example in the dev set to determine thresholds that maximize either the EM score or the F1 score. It then recomputes the best possible EM score and F1 score associated with that null threshold. 

In [139]:
# The default threshold is set to 1.0 -- we'll leave it there for now
results_default_thresh = squad_evaluate(examples, 
                                        preds, 
                                        no_answer_probs=null_odds, 
                                        no_answer_probability_threshold=1.0)

In [140]:
results_default_thresh

OrderedDict([('exact', 66.25958056093658),
             ('f1', 69.66994428499025),
             ('total', 11873),
             ('HasAns_exact', 68.91025641025641),
             ('HasAns_f1', 75.74076391627662),
             ('HasAns_total', 5928),
             ('NoAns_exact', 63.61648444070648),
             ('NoAns_f1', 63.61648444070648),
             ('NoAns_total', 5945),
             ('best_exact', 68.36519834919565),
             ('best_exact_thresh', -4.189256191253662),
             ('best_f1', 71.1144383018176),
             ('best_f1_thresh', -3.767639636993408)])

The first three blocks have identical values as our first evaluation because they are based on the default threshold (which is currently 1.0). However, the values in the fourth block have been updated by taking into account the `null_odds` information.  When a given example's `score_diff` is greater than the threshold, the prediction is flipped to a null answer which affects the overall EM and F1 scores. 

Let's use the `best_f1_thresh` and run the evaluation once more one more so that we can see a breakdown of how our model does on `HasAns` and `NoAns` examples:

In [151]:
best_f1_thresh = -3.767639636993408
results_f1_thresh = squad_evaluate(examples, 
                                   preds, 
                                   no_answer_probs=null_odds, 
                                   no_answer_probability_threshold=best_f1_thresh)

In [150]:
results_f1_thresh

OrderedDict([('exact', 68.31466352227744),
             ('f1', 71.11443830181771),
             ('total', 11873),
             ('HasAns_exact', 61.53846153846154),
             ('HasAns_f1', 67.14604014127524),
             ('HasAns_total', 5928),
             ('NoAns_exact', 75.07148864592094),
             ('NoAns_f1', 75.07148864592094),
             ('NoAns_total', 5945),
             ('best_exact', 68.36519834919565),
             ('best_exact_thresh', -4.189256191253662),
             ('best_f1', 71.1144383018176),
             ('best_f1_thresh', -3.767639636993408)])

When we used the default threshold of 1.0, we saw that our `NoAns_f1` score was a mere 63.6 but when we use the `best_f1_thresh`, we now get a `NoAns_f1` score of 75! Nearly a 12 point jump! The downside is that we lose some ground in how well our model correctly predicts `HasAns` examples. Overall, however, we see a net increase of a couple points in both EM and F1 scores. This demonstrates that the computing null scores and properly using a null threshold  significantly increases QA performance on the SQuAD2.0 dev set with almost no additional work.

# Putting it all together

Below we have a new method that will select more robust predictions, compute scores for the best text predictions as well as for the null prediction, and will use these scores along with a threshold to determine whether the question should be answered. As a bonus, this method also computes and returns the probability of the answer which is often easier to interpret than a logit score. The probability of a prediction will depend on `nbest` since they are computed with a softmax. 

In [147]:
import numpy as np

def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum()

def get_robust_prediction(example, nbest=10, null_threshold=1.0):
    # given a question id (qas_id or qid), load the example, get the model outputs and generate an answer
    question = example.question_text
    context = example.context_text

    inputs = tokenizer.encode_plus(question, context, return_tensors='pt')

    start_logits, end_logits = model(**inputs)

    # convert our start and end logit tensors to lists
    start_logits = to_list(start_logits)[0]
    end_logits = to_list(end_logits)[0]

    # sort our start and end logits from largest to smallest, keeping track of the index
    start_idx_and_logit = sorted(enumerate(start_logits), key=lambda x: x[1], reverse=True)
    end_idx_and_logit = sorted(enumerate(end_logits), key=lambda x: x[1], reverse=True)
    
    start_indexes = [idx for idx, logit in start_idx_and_logit[:5]]
    end_indexes = [idx for idx, logit in end_idx_and_logit[:5]]

    # convert the token ids from a tensor to a list
    tokens = to_list(inputs['input_ids'])[0]

    # question tokens are defined as those between the CLS token (101, at position 0) and first SEP (102) token 
    question_indexes = [i+1 for i, token in enumerate(tokens[1:tokens.index(102)])]
    question_indexes
    
    # keep track of all preliminary predictions
    PrelimPrediction = collections.namedtuple(  # pylint: disable=invalid-name
        "PrelimPrediction", ["start_index", "end_index", "start_logit", "end_logit"]
    )

    prelim_preds = []
    for start_index in start_indexes:
        for end_index in end_indexes:
            # throw out invalid predictions
            if start_index in question_indexes:
                continue
            if end_index in question_indexes:
                continue
            if end_index < start_index:
                continue
            prelim_preds.append(
                PrelimPrediction(
                    start_index = start_index,
                    end_index = end_index,
                    start_logit = start_logits[start_index],
                    end_logit = end_logits[end_index]
                )
            )

    # keep track of all best predictions
    # This will be the pool from which answer probabilities are computed 
    BestPrediction = collections.namedtuple(  # pylint: disable=invalid-name
        "BestPrediction", ["text", "start_logit", "end_logit"]
    )

    nbest_predictions = []
    seen_predictions = []
    for pred in prelim_preds:
        if len(nbest_predictions) >= nbest: 
            break
        if pred.start_index > 0: # non-null answers have start_index > 0
            text = tokenizer.convert_tokens_to_string(
                tokenizer.convert_ids_to_tokens(
                    tokens[pred.start_index:pred.end_index+1]
                )
            )
            # Clean whitespace
            text = text.strip()
            text = " ".join(text.split())

            if text in seen_predictions:
                continue

            # flag this text as being seen -- if we see it again, no need to add it to the list
            seen_predictions.append(text) 

            # add this text prediction to a pruned list of the top nbest predictions
            nbest_predictions.append(BestPrediction(text=text, 
                                                    start_logit=pred.start_logit, 
                                                    end_logit=pred.end_logit))
        
    # Add the null prediction
    nbest_predictions.append(BestPrediction(text="", start_logit=start_logits[0], end_logit=end_logits[0]))

    # Compute the probability of each prediction - this is not needed but nice to have
    all_scores = [pred.start_logit+pred.end_logit for pred in nbest_predictions] 
    probabilities = softmax(np.array(all_scores))
        
    # compute the null score as the sum of the [CLS] token logits
    score_null = nbest_predictions[-1].start_logit + nbest_predictions[-1].end_logit
    score_non_null = nbest_predictions[0].start_logit + nbest_predictions[0].end_logit

    # compute the difference between the null score and the best non-null score
    score_diff = score_null - score_non_null

    # If that difference is greater than the null threshold, return the null answer
    if score_diff > null_threshold:
        return "", probabilities[-1]
    else:
        return nbest_predictions[0].text, probabilities[0]
    

Do we now get the right answer (an empty string) for that tricky no-answer example we were working with? 

In [148]:
get_robust_prediction(example, nbest=10, null_threshold=best_f1_thresh)

('', 0.34837593510791465)

Woohoo!! We got the right answer this time!! 

Even if we didn't have the best threshold in place, our additional checks still allow us to output more sensible looking answers, rejecting predictions that include the question tokens. 

In [152]:
get_robust_prediction(example, nbest=10, null_threshold=1.0)

('free oxygen began to outgas from the oceans', 0.4293458323979202)

And if it hadn't been a trick question, this would be the correct answer! Seems like distilBERT could use some improvement in number understanding. 

Using a robust prediction method like the above will do more than allow your model to perform better on a curated dev set, though this is an important first step. It will also provide your model with a slightly better ability to refrain from answering questions that simply don't have an answer in the associated passage. This is a crucial feature for QA models because it's not enough to get an answer if that answer doesn't make sense for the question we ask. We want our models to tell us something useful -- and sometimes that means telling us nothing at all. 